In [10]:
import wrappers
import dqn_model

import argparse
import time
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.optim as optim




DEFAULT_ENV_NAME = "Br"
MEAN_REWARD_BOUND = 20.5

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02


Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])


class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)


class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward


def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device=device, dtype=torch.int64)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.tensor(dones).to(device=device, dtype=torch.bool)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)


if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument("--cuda", default=False, action="store_true", help="Enable cuda")
    #parser.add_argument("--env", default=DEFAULT_ENV_NAME,
                        #help="Name of the environment, default=" + DEFAULT_ENV_NAME)
    #parser.add_argument("--reward", type=float, default=MEAN_REWARD_BOUND,
                        #help="Mean reward boundary for stop of training, default=%.2f" % MEAN_REWARD_BOUND)
    #args = parser.parse_args()
    
    import easydict
    args = easydict.EasyDict({
            "no_cuda": False,
            "env": DEFAULT_ENV_NAME,
            "reward": MEAN_REWARD_BOUND,
    })
    
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    env = wrappers.make_env(args.env)

    net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    tgt_net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
   
    print(net)

    buffer = ExperienceBuffer(REPLAY_SIZE)
    agent = Agent(env, buffer)
    epsilon = EPSILON_START

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    best_mean_reward = None

    while True:
        frame_idx += 1
        epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)
            speed = (frame_idx - ts_frame) / (time.time() - ts)
            ts_frame = frame_idx
            ts = time.time()
            mean_reward = np.mean(total_rewards[-100:])
            print("%d: done %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" % (
                frame_idx, len(total_rewards), mean_reward, epsilon,
                speed
            ))

            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), args.env + "-best.dat")
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
                best_mean_reward = mean_reward
            if mean_reward > args.reward:
                print("Solved in %d frames!" % frame_idx)
                break

        if len(buffer) < REPLAY_START_SIZE:
            continue

        if frame_idx % SYNC_TARGET_FRAMES == 0:
            tgt_net.load_state_dict(net.state_dict())

        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        loss_t = calc_loss(batch, net, tgt_net, device=device)
        loss_t.backward()
        optimizer.step()



DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=4, bias=True)
  )
)
128: done 1 games, mean reward 0.000, eps 1.00, speed 108.97 f/s
274: done 2 games, mean reward 0.000, eps 1.00, speed 105.95 f/s
396: done 3 games, mean reward 0.000, eps 1.00, speed 9.84 f/s
617: done 4 games, mean reward 0.500, eps 0.99, speed 120.59 f/s
Best mean reward updated 0.000 -> 0.500, model saved
787: done 5 games, mean reward 0.600, eps 0.99, speed 118.39 f/s
Best mean reward updated 0.500 -> 0.600, model saved
937: done 6 games, mean reward 0.667, eps 0.99, speed 116.24 f/s
Best mean reward updated 0.600 -> 0.667, model saved
1103: done 7 games, mean reward 0

KeyboardInterrupt: 

In [1]:
torch.save(net.state_dict(),'Pongintelligence')

In [5]:
torch.load('Pongintelligence')

OrderedDict([('conv.0.weight',
              tensor([[[[ 1.3823e-02,  2.3134e-02,  2.7223e-02,  ...,  1.4439e-01,
                         -3.6750e-02, -5.7741e-02],
                        [ 3.2420e-02,  1.6099e-01,  1.1500e-01,  ...,  1.8085e-01,
                          8.3257e-02, -8.8230e-03],
                        [-3.4807e-02,  1.0339e-01, -5.2026e-02,  ...,  6.8062e-02,
                         -2.9262e-02,  3.5681e-02],
                        ...,
                        [-2.7923e-02, -1.1591e-02,  4.9748e-02,  ..., -4.3495e-03,
                         -4.1110e-02, -2.7610e-02],
                        [-6.0268e-02, -2.1677e-02,  4.8180e-02,  ...,  6.9722e-02,
                         -6.9364e-03, -1.3889e-01],
                        [-4.5784e-02,  7.5032e-02,  1.1316e-02,  ...,  3.7016e-02,
                          1.3416e-01, -6.6579e-02]],
              
                       [[-7.5889e-02, -3.2661e-02, -3.9822e-02,  ...,  9.5667e-02,
                          1.087

In [7]:
x.env.observation_space

Box(210, 160, 3)

In [8]:
x.env.action_space

Discrete(6)